# Virtual Frequency Visualization Code

### This notebook can be used to produce the virtual frequency visualizations shown in the manuscript and in the supplementary information.

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from Freq import butterworth

In [ ]:
def compute_virtual_eoe(eoe):
    if torch.is_tensor(eoe):
        return torch.fft.ifftshift(torch.fft.fft2(torch.abs(torch.fft.fftshift(torch.fft.fft2(eoe)))**2))/(eoe.shape[0]*eoe.shape[1])
    else:        
        eoe_far_field = np.abs(np.fft.fftshift(np.fft.fft2(eoe)))**2.0
        return np.fft.ifftshift(np.fft.fft2(eoe_far_field))

In [ ]:
cmap_setting = 'pink'
vmax_setting = 28000

# Define the Frequency Filters

In [ ]:
etendue_expansion_factor = 4; butter_fac = 1; butter_order = 5; R = 1024; C = 1024
butter_4 = butterworth(C, R, int(np.sqrt(etendue_expansion_factor)//butter_fac), order=butter_order)

etendue_expansion_factor = 16; butter_fac = 1; butter_order = 5; R = 2048; C = 2048
butter_16 = butterworth(C, R, int(np.sqrt(etendue_expansion_factor)//butter_fac), order=butter_order)

etendue_expansion_factor = 36; butter_fac = 1; butter_order = 5; R = 3072; C = 3072
butter_36 = butterworth(C, R, int(np.sqrt(etendue_expansion_factor)//butter_fac), order=butter_order)

etendue_expansion_factor = 64; butter_fac = 1; butter_order = 5; R = 4096; C = 4096
butter_64 = butterworth(C, R, int(np.sqrt(etendue_expansion_factor)//butter_fac), order=butter_order)

plt.figure(figsize=(16,10))
plt.subplot(141), plt.imshow(butter_4, vmin=0, vmax=1, cmap='pink')
plt.subplot(142), plt.imshow(butter_16, vmin=0, vmax=1, cmap='pink')
plt.subplot(143), plt.imshow(butter_36, vmin=0, vmax=1, cmap='pink')
plt.subplot(144), plt.imshow(butter_64, vmin=0, vmax=1, cmap='pink')

# Display Virtual Frequency of Natural Images

### These frequencies are computed as an average over a dataset of natural images.

In [ ]:
freq_avg_4 = np.load('./Virtual_Freq/natural_images_4x.npy')
freq_avg_16 = np.load('./Virtual_Freq/natural_images_16x.npy')
freq_avg_36 = np.load('./Virtual_Freq/natural_images_36x.npy')
freq_avg_64 = np.load('./Virtual_Freq/natural_images_64x.npy')

scale_4 = np.sum(butter_64 * freq_avg_64) / np.sum(butter_4 * freq_avg_4)
freq_avg_4 = freq_avg_4 * scale_4
scale_16 = np.sum(butter_64 * freq_avg_64) / np.sum(butter_16 * freq_avg_16)
freq_avg_16 = freq_avg_16 * scale_16
scale_36 = np.sum(butter_64 * freq_avg_64) / np.sum(butter_36 * freq_avg_36)
freq_avg_36 = freq_avg_36 * scale_36

freq_avg_4_butter = freq_avg_4 * butter_4
freq_avg_16_butter = freq_avg_16 * butter_16
freq_avg_36_butter = freq_avg_36 * butter_36
freq_avg_64_butter = freq_avg_64 * butter_64

plt.figure(figsize=(16,10))
plt.subplot(141), plt.imshow(freq_avg_4_butter, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(142), plt.imshow(freq_avg_16_butter, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(143), plt.imshow(freq_avg_36_butter, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(144), plt.imshow(freq_avg_64_butter, vmin=0, vmax=vmax_setting, cmap=cmap_setting)

In [ ]:
# Display center of the virtual frequency

center_width = 1024 // 8
center_side = (1024 - center_width) // 2
freq_avg_4_center = freq_avg_4_butter[center_side:-center_side,center_side:-center_side]

center_width = 2048 // 8
center_side = (2048 - center_width) // 2
freq_avg_16_center = freq_avg_16_butter[center_side:-center_side,center_side:-center_side]

center_width = 3072 // 8
center_side = (3072 - center_width) // 2
freq_avg_36_center = freq_avg_36_butter[center_side:-center_side,center_side:-center_side]

center_width = 4096 // 8
center_side = (4096 - center_width) // 2
freq_avg_64_center = freq_avg_64_butter[center_side:-center_side,center_side:-center_side]

plt.figure(figsize=(16,10))
plt.subplot(141), plt.imshow(freq_avg_4_center, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(142), plt.imshow(freq_avg_16_center, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(143), plt.imshow(freq_avg_36_center, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(144), plt.imshow(freq_avg_64_center, vmin=0, vmax=vmax_setting, cmap=cmap_setting)

# Display Virtual Frequency of the Neural Étendue Expanders

### Note that here we are computing the virtual frequency on the neural étendue expanders before any fabrication quantization constraint is applied.

In [ ]:
phase = torch.load('./Virtual_Freq/neural_mono_float_4x.pth')
phase = phase.numpy()
eoe_4 = np.exp(1j*phase)
veoe_4 = np.abs(compute_virtual_eoe(eoe_4))
scale_4 = np.sum(butter_64 * freq_avg_64) / np.sum(butter_4 * veoe_4)
veoe_4 = veoe_4 * scale_4

phase = torch.load('./Virtual_Freq/neural_mono_float_16x.pth')
phase = phase.numpy()
eoe_16 = np.exp(1j*phase)
veoe_16 = np.abs(compute_virtual_eoe(eoe_16))
scale_16 = np.sum(butter_64 * freq_avg_64) / np.sum(butter_16 * veoe_16)
veoe_16 = veoe_16 * scale_16

phase = torch.load('./Virtual_Freq/neural_mono_float_36x.pth')
phase = phase.numpy()
eoe_36 = np.exp(1j*phase)
veoe_36 = np.abs(compute_virtual_eoe(eoe_36))
scale_36 = np.sum(butter_64 * freq_avg_64) / np.sum(butter_36 * veoe_36)
veoe_36 = veoe_36 * scale_36

phase = torch.load('./Virtual_Freq/neural_mono_float_64x.pth')
phase = phase.numpy()
eoe_64 = np.exp(1j*phase)
veoe_64 = np.abs(compute_virtual_eoe(eoe_64))
scale_64 = np.sum(butter_64 * freq_avg_64) / np.sum(butter_64 * veoe_64)
veoe_64 = veoe_64 * scale_64

plt.figure(figsize=(16,10))
plt.subplot(141), plt.imshow(veoe_4, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(142), plt.imshow(veoe_16, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(143), plt.imshow(veoe_36, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(144), plt.imshow(veoe_64, vmin=0, vmax=vmax_setting, cmap=cmap_setting)

In [ ]:
# Display center of the virtual frequency

center_width = 1024 // 8
center_side = (1024 - center_width) // 2
veoe_4_center = veoe_4[center_side:-center_side,center_side:-center_side]
center_width = 2048 // 8
center_side = (2048 - center_width) // 2
veoe_16_center = veoe_16[center_side:-center_side,center_side:-center_side]
center_width = 3072 // 8
center_side = (3072 - center_width) // 2
veoe_36_center = veoe_36[center_side:-center_side,center_side:-center_side]
center_width = 4096 // 8
center_side = (4096 - center_width) // 2
veoe_64_center = veoe_64[center_side:-center_side,center_side:-center_side]

plt.figure(figsize=(16,10))
plt.subplot(141), plt.imshow(veoe_4_center, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(142), plt.imshow(veoe_16_center, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(143), plt.imshow(veoe_36_center, vmin=0, vmax=vmax_setting, cmap=cmap_setting)
plt.subplot(144), plt.imshow(veoe_64_center, vmin=0, vmax=vmax_setting, cmap=cmap_setting)